In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import app_functions as af
import plots as pl

import pandas as pd
import plotly.express as px
import numpy as np

2023-06-29 09:18:42.521 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-06-29 09:18:42.526 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-06-29 09:18:42.529 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-06-29 09:18:42.532 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-06-29 09:18:42.535 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-06-29 09:18:42.539 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-06-29 09:18:42.724 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-06-29 09:18:42.727 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-06-

In [102]:
#data_dir_fp = r'Z:\Interés Departamental\Model Portfolio\Hugo\Ranking DIP European Equities\copia 12-04-2023\data\vertical_dowload_files'
#prices_csv_filepath = r'Z:\Interés Departamental\Model Portfolio\Hugo\Ranking DIP European Equities\copia 12-04-2023\data\vertical_dowload_files\PriceClose.csv'
#mask_fp = r'Z:\Interés Departamental\Model Portfolio\Hugo\Ranking DIP European Equities\copia 12-04-2023\data\PriceClose_vertical\monthly_constituents_filter.csv'
data_dir_fp = r'Z:\Interés Departamental\Model Portfolio\Laboratorio de Ideas\Python\datos Hugo'
prices_csv_filepath = r'Z:\Interés Departamental\Model Portfolio\Laboratorio de Ideas\Python\datos Hugo\PriceClose.csv'
mask_fp = r"Z:\Interés Departamental\Model Portfolio\Laboratorio de Ideas\Python\Filtros\stoxx600_month_end_mask.csv"

prices_df = af.read_and_sort_data(prices_csv_filepath)
mask = af.read_and_sort_data(mask_fp)
data_dict = af.filter_data(data_dir_fp,0.8,0.8,600,mask)[0]

pe_df = data_dict['PE']

Filtering:  CompanyMarketCapitalization.csv
Filtering:  EPSMean.csv
Filtering:  EV.csv
Filtering:  EVTOEBIT.csv
Filtering:  EVTOEBITDA.csv
Filtering:  EVTOOperatingCashFlow.csv
Filtering:  EVToSales.csv
Filtering:  HistPEG.csv
Filtering:  NetDebtToEBITDA.csv
Filtering:  NetDebtToEV.csv
Filtering:  PE.csv
Filtering:  PriceClose.csv
Filtering:  PriceToBVPerShare.csv
Filtering:  PriceToCFPerShare.csv
Filtering:  PriceToSalesPerShare.csv
Filtering:  PriceToTangBVPerShare.csv
Filtering:  ROCEActValue.csv
Filtering:  TotalDebtToEBITDA.csv
Filtering:  TotalDebtToEV.csv


In [4]:
searching_df = pe_df.loc['2007-01-01':'2010-12-01']

In [100]:
def infer_inferred_freqs(data:pd.DataFrame|pd.Series) -> str:
    try:
        all_samples = np.lib.stride_tricks.sliding_window_view(data.index, window_shape=10)
    except ValueError as x:
        if x.args[0] == 'window shape cannot be larger than input array shape':
            raise ValueError('Insufficient data to infer frequency. Must be longer than 10')
        raise x
    
    rand_indices = np.unique(np.random.randint(0, all_samples.shape[0], size=30))

    samples = all_samples[rand_indices]

    freqs = pd.Series(list(samples)).apply(pd.infer_freq)

    freqs_count = freqs.value_counts().sort_values(ascending=False)

    if len(freqs_count) == 0:
        raise ValueError('Inferred data frequency is None')
    elif len(freqs_count)>1:
        if freqs_count.duplicated(keep='first').iloc[1]==True:
            raise ValueError('Inferred data frequency is Duplicated')

    return freqs_count.index[0]

In [115]:
df = data_dict['PE']#[list(data_dict.keys())[5]]
relative = True

infered_freq = infer_inferred_freqs(df)

filled_df = df.asfreq(infered_freq)

if relative == True:
    na_counts = filled_df.notna().sum(axis=1)/len(filled_df.columns)
else:
    na_counts = filled_df.notna().sum(axis=1)

changes = (na_counts==0).astype(int).diff()

na_ranges = np.array((changes.iloc[changes.index.get_indexer(changes.loc[changes==1].index)-1].index,changes.loc[changes==-1].index)).T

fig = px.line(na_counts)

for start, end in na_ranges.astype(str):

    fig.add_vrect(x0=start, x1=end, fillcolor='red', opacity=0.5, line_width=0)

fig.show()

In [ ]:
filled_pe_df= pe_df.asfreq(infered_freq)

In [ ]:
na_counts = filled_pe_df.notna().sum(axis=1)

In [ ]:
changes = (na_counts==0).astype(int).diff()

In [ ]:
na_ranges = np.array((changes.iloc[changes.index.get_indexer(changes.loc[changes==1].index)-1].index,changes.loc[changes==-1].index)).T

In [ ]:
fig = px.line(na_counts)

for start, end in na_ranges.astype(str):
    print(start, type(start))
    fig.add_vrect(x0=start, x1=end, fillcolor='red', opacity=0.5, line_width=0)

fig.show()